In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from multiprocessing.pool import ThreadPool
import time


In [37]:
def cnt_from_img(img, num, centers=False):
    # Load the image
    im = cv2.imread(img)
    # Crop the 4096x2160 image to 
    im = im[200:700, 1698:2398]
    # Convert to greyscale
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    # Denoise
    denoise = cv2.fastNlMeansDenoising(imgray, templateWindowSize = 7, searchWindowSize=21, h=12)
    # Blur
    #blur = cv2.bilateralFilter(denoise,9,500,250)
    blur = cv2.GaussianBlur(denoise,(5,5),0)
    # Thresholding
    #thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,19,2)#+cv2.THRESH_OTSU)
    thresh = cv2.adaptiveThreshold(blur, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,19,2)
    # Detect edges
    edges = cv2.Canny(thresh, 100,100)
    #edge_name = "data/BP02_midsag/edges/e_%i.bmp"%num
    #cv2.imwrite(edge_name, thresh)
    # Get contours
    im2, contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # Sort contours by area
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    # Calculate moments and areas
    moments = []
    areas = []
    for c in contours:
        # Calculate the centroid of the contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            moments.append(M)
        if centers:
            cv2.circle(im, (cX, cY), 2, (255, 0, 0), -1)
        # Calculate the area of the contour
        A = cv2.contourArea(c)
        areas.append(A)
        
 
    # Display
    cv2.drawContours(im, contours, -1, (0, 255, 0), 1)
    output_name = 'data/BP02_midsag/contours/cnt_%i.bmp' % num
    cv2.imwrite(output_name, im)


In [19]:
path = 'data/BP02_midsag/BMP/'
out_path = 'data/BP02_midsag/cnn/'

In [36]:
p = ThreadPool(8)
images = [(path + 'BMP%05d.bmp' % i, i) for i in range(30,119)]
start = time.time()
r = p.starmap(cnt_from_img, images)
end = time.time()
print(end-start)

12.286837577819824


In [5]:
p = ThreadPool(8)
def bulk_crop(img):
    # Load the image
    im = cv2.imread(img)
    # Crop the 4096x2160 image to 
    im = im[210:710, 1798:2298]
    # Convert to greyscale
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    output_name = 'data/BP02_midsag/crop/cr_%i.bmp' % num
    cv2.imwrite(output_name, imgray)
    
    
def stddev_for_pix(x,y,std_arr,images):
    std_arr[x,y] = np.std(images[:,x,y]
                        )
    
def pix_int_stddev(image_stack):
    if not image_stack:
        print("image_stack is empty")
        return
    
    #open images
    images = []
    for image in image_stack:
        im = cv2.imread(image,0)
        #grey = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        images.append(im)
        
    images = np.asarray(images)
    l, w = np.shape(images[0])
    std_dev = np.zeros(np.shape(images[0]))
    #for x in range(0, l):
    #    for y in range(0,w):
    #        std_dev[x, y] = np.std(images[:,x,y])
            
    args = [(x,y,std_dev,images) for x in range(0,l) for y in range(0,w)]
    _ = p.starmap(stddev_for_pix, args)
    return std_dev

In [6]:
#path = 'data/BP02_midsag/BMP/'
#for i in range(30,121):
#    image = path + 'BMP%05d.bmp' % i
#    crop(image, i)
path = 'data/BP02_midsag/crop/'
start_image = 30
end_image = 120
img_range = end_image-start_image
stddev_gran = 2 # use 5 images to compute stddev images
num_steps = int(img_range/stddev_gran)
for i in range(0,num_steps):
    start_idx = i*stddev_gran + start_image
    end_idx = start_idx + stddev_gran
    images = [path + 'cr_%i.bmp' % i for i in range(start_idx,end_idx)]
    im = pix_int_stddev(images)

    output_name = 'data/BP02_midsag/std_dev/std_dev_%i.jpg' % i
    plt.pcolormesh(im)
    plt.savefig(output_name)
    plt.clf()

ValueError: not enough values to unpack (expected 2, got 0)